In [1]:
import wandb

In [192]:
import seaborn as sns
import pandas as pd
import time
import cv2
import os
import albumentations
import timm
from pytorch_lightning.loggers import WandbLogger
import pickle
from sklearn.model_selection import StratifiedKFold
from pytorch_lightning.callbacks import ModelCheckpoint,EarlyStopping
from torchvision.models import resnet50,resnet18
from sklearn.preprocessing import LabelEncoder,MultiLabelBinarizer
from albumentations.pytorch import ToTensorV2
import json
import torch.nn.functional as F
import numpy as np
import random
import pytorch_lightning as pl
import torch.optim as optim
import torch.nn as nn
import torchmetrics
from facenet_pytorch import MTCNN,extract_face
from torch.utils.data import Dataset,DataLoader
import torch
from sklearn.utils.class_weight import compute_class_weight


In [3]:
def seeding(seed=2021):
    np.random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    torch.random.manual_seed(seed)
    random.seed(seed)
seeding()

In [4]:
data = pd.read_csv("UTK.csv")

In [10]:
max(data.age)

116

In [16]:
class cfg:
    img_size=256
    max_epochs=100
    model_name = "resnet18"
    patience = [5,2]
    factor= .1
    folds=5
    min_lr=1e-8
    logger = WandbLogger(project="age_gend")

In [198]:
class age_gend_dataset(Dataset):
    def __init__(self,df,transforms):
        self.df = df
        self.transforms = transforms
        #self.root_dir = root_dir
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self,idx):
        img = cv2.imread(self.df.loc[idx,'path'])
        #print(img.shape)
        age = self.df.loc[idx,'age']
        gend = self.df.loc[idx,'gender']
        assert img is not None
        if self.transforms is not None:
            img = self.transforms(image=img)['image']

        sample = {'image':img,'age':torch.Tensor([age])/116,'gender':torch.Tensor([gend])}
        return sample

In [199]:
class MTL_Loss(nn.Module):
    def __init__(self,task_num):
        super(MTL_Loss,self).__init__()
        self.task_num = task_num
        self.log_vars = nn.Parameter(torch.zeros((self.task_num)))
    def forward(self,pred_age,pred_gend,tar_gend,tar_age):
        

        loss0 = nn.functional.binary_cross_entropy_with_logits(pred_gend,tar_gend)
        loss1 = nn.functional.mse_loss(pred_age,tar_age)

        precision0 = torch.exp(-self.log_vars[0])
        loss0 = precision0*loss0 + self.log_vars[0]

        precision1 = torch.exp(-self.log_vars[1])
        loss1 = precision1*loss1 + self.log_vars[1]

        return loss0+loss1




In [200]:
mtl = MTL_Loss(task_num=2)

In [201]:
def train_augs():
    return albumentations.Compose([
       albumentations.RandomResizedCrop(cfg.img_size, cfg.img_size, scale=(0.9, 1), p=1), 
       albumentations.HorizontalFlip(p=0.5),
       albumentations.VerticalFlip(p=0.5),
       albumentations.ShiftScaleRotate(p=0.5),
       albumentations.HueSaturationValue(hue_shift_limit=10, sat_shift_limit=10, val_shift_limit=10, p=0.7),
       albumentations.RandomBrightnessContrast(brightness_limit=(-0.2,0.2), contrast_limit=(-0.2, 0.2), p=0.7),
       albumentations.CLAHE(clip_limit=(1,4), p=0.5),
       albumentations.OneOf([
           albumentations.OpticalDistortion(distort_limit=1.0),
           albumentations.GridDistortion(num_steps=5, distort_limit=1.),
           albumentations.ElasticTransform(alpha=3),
       ], p=0.2),
       albumentations.OneOf([
           albumentations.GaussNoise(var_limit=[10, 50]),
           albumentations.GaussianBlur(),
           albumentations.MotionBlur(),
           albumentations.MedianBlur(),
       ], p=0.2),
      albumentations.Resize(cfg.img_size, cfg.img_size),
      albumentations.OneOf([
          albumentations.JpegCompression(),
          albumentations.Downscale(scale_min=0.1, scale_max=0.15),
      ], p=0.2),
      albumentations.IAAPiecewiseAffine(p=0.2),
      albumentations.IAASharpen(p=0.2),
      albumentations.Cutout(max_h_size=int(cfg.img_size * 0.1), max_w_size=int(cfg.img_size * 0.1), num_holes=5, p=0.5),
      albumentations.Normalize(p=1.0),
      ToTensorV2()
    ])
    


In [202]:
class AgeGendModel(nn.Module):
    def __init__(self):
        super(AgeGendModel,self).__init__()
        if 'resnet18' in cfg.model_name:
            self.model = eval(cfg.model_name)(pretrained=False)
            for params in self.model.parameters():
                params.require_grad = True
            self.model = nn.Sequential(*list(self.model.children())[:-2])
            self.avg_pool = nn.AdaptiveAvgPool2d(output_size=(1,1))
        
            self.clf = nn.Linear(512,1)
            self.reg = nn.Linear(512,1)
    
    def forward(self,x):
        x = self.model(x)
        x = self.avg_pool(x)
        x = x.view(x.size(0),-1)
        #print(x.shape)
        #x = self.lin1(x)
        #x = self.lin2(x)

        gend = self.clf(x)
        age = torch.sigmoid(self.reg(x))


        return gend,age



In [203]:
model2 = AgeGendModel()

## Model Class

In [204]:
class AgeGendNet(pl.LightningModule):
    def __init__(self):
        super(AgeGendNet,self).__init__()

        self.model = AgeGendModel()
        

    def forward(self,x):
        return self.model(x)
    
    def configure_optimizers(self):
        op = optim.Adam(self.model.parameters(),lr=0.01)
        
        scheduler = {
            'scheduler':optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer=op,T_0=10),
            'monitor':'val_loss',
            'interval':'epoch',
            'frequency':1,
            'strict':True
        }

        self.op = op
        self.scheduler = scheduler
        
        return [op],[scheduler]

    def training_step(self,batch,batch_idx):
        y_hat_gend,y_hat_age = self.model(batch['image'])
        loss_tr = mtl(pred_gend=y_hat_gend,pred_age=y_hat_age,tar_gend=batch['gender'],tar_age=batch['age'])
        #f1_tr = torchmetrics.functional.accuracy(y_hat_gend.sigmoid(),batch['gender']) 
        #mae_tr = torchmetrics.functional.mean_absolute_error(y_hat_age.relu(),batch['age'])
        self.log("TrainLoss",loss_tr,prog_bar=True,on_step=False,on_epoch=True)
        return loss_tr
    
    def validation_step(self,batch,batch_idx):
        y_hat_gend,y_hat_age = self.model(batch['image'])
        loss_val = mtl(pred_gend=y_hat_gend,pred_age=y_hat_age,tar_gend=batch['gender'],tar_age=batch['age'])
        #f1_val = torchmetrics.functional.accuracy(y_hat_gend.sigmoid(),batch['gender'])
        #mae_val = torchmetrics.functional.mean_absolute_error(y_hat_age.relu(),batch['age'])
        self.log("val_loss",loss_val,prog_bar=True,on_step=False,on_epoch=True)
        return loss_val

In [205]:
model = AgeGendNet()

In [206]:
train_data = pd.read_csv("UTK.csv")

In [209]:
def main():
    
    i=0

    estp = EarlyStopping(monitor="val_loss",mode="min",patience=cfg.patience[0])

    if os.path.isdir(f"{cfg.model_name}_{cfg.img_size}_Model_Fold-{i+1}-2"):
        print("Already Exists")
    else:
        os.makedirs(f"{cfg.model_name}_{cfg.img_size}_Model_Fold-{i+1}-2")
    

    mod_ckpt = ModelCheckpoint(
    monitor='val_loss',
    mode='min',
    dirpath=f"{cfg.model_name}_{cfg.img_size}_Model_Fold-{i+1}-2",
    filename='Resnet50_256_Checkpoint-ValLoss:{val_loss:.4f}',
    save_top_k=1,
)
    trainer = pl.Trainer(gpus=1,precision=16,max_epochs=cfg.max_epochs,progress_bar_refresh_rate=30,deterministic=True,benchmark=True,callbacks=[mod_ckpt,estp],logger=cfg.logger,accumulate_grad_batches=1)
    
    
    print(f"Initializing model Fold - {i+1}/5")
    model = AgeGendNet()
    print("*** Model Initialization Completed ***")
    train_recs = train_data[train_data.folds != i].reset_index(drop=True)
    val_recs = train_data[train_data.folds == i].reset_index(drop=True)

    train_dataset = age_gend_dataset(df=train_recs,transforms=train_augs())
    val_dataset = age_gend_dataset(df=val_recs,transforms=val_augs())

    train_loader = DataLoader(train_dataset,batch_size=32,num_workers=4)
    val_loader = DataLoader(val_dataset,batch_size=32,num_workers=4)

    trainer.fit(model,train_loader,val_loader)

In [210]:
main()

eGendModel | 11.2 M
---------------------------------------
11.2 M    Trainable params
0         Non-trainable params
11.2 M    Total params
44.710    Total estimated model params size (MB)
Initializing model Fold - 1/5
*** Model Initialization Completed ***
Epoch 0:  81%|████████  | 600/742 [02:13<00:31,  4.51it/s, loss=0.694, v_num=li8g, val_loss=1.040]
Validating: 0it [00:00, ?it/s]
Epoch 1:  81%|████████  | 600/742 [02:14<00:31,  4.45it/s, loss=0.679, v_num=li8g, val_loss=0.652, TrainLoss=0.728]
Validating: 0it [00:00, ?it/s]
Epoch 2:  81%|████████  | 600/742 [02:11<00:31,  4.56it/s, loss=0.637, v_num=li8g, val_loss=0.643, TrainLoss=0.669]
Validating: 0it [00:00, ?it/s]
Epoch 3:  81%|████████  | 600/742 [02:11<00:31,  4.57it/s, loss=0.613, v_num=li8g, val_loss=0.622, TrainLoss=0.645]
Validating: 0it [00:00, ?it/s]
Epoch 4:  81%|████████  | 600/742 [02:13<00:31,  4.51it/s, loss=0.591, v_num=li8g, val_loss=0.564, TrainLoss=0.621]
Validating: 0it [00:00, ?it/s]
Epoch 5:  81%|████████ 

928,  1447.9056, 15595.3271,
                       1399.6440, 12397.3584,  4446.0747,  4913.3857, 11369.0117,   972.9417,
                        384.1671,  6722.8203,  8443.7656,   829.0696,   928.3211,  1049.8445,
                        255.9720, 10292.6934, 15058.1182,  2261.2695,  2094.4441,  5347.5503,
                       4247.1079,   210.3136,  2566.8687,  1486.0464,  3581.6904,   601.6384,
                      11875.2842,   196.8144,  5330.7832,   720.2455,  1410.9438,  2394.3589,
                       1500.3696,  4117.3574,   698.1847,  9151.6299,  6040.6343,  4428.8623,
                       3508.2937,  6104.7402,  2192.2209,  8163.5342,   448.6442, 16883.4863,
                       3584.3674,  6412.1675,  3056.7168,  4067.2166,  4326.5020,  1516.6954,
                        506.0337,  8971.5098,  7985.8037,  3375.4756,  5400.2642,  8261.2627,
                      11316.9453, 19741.1152,  2599.9685,  8815.8770,  1531.7903,  7633.0493,
                       3771.569

In [194]:
model.eval()
img = cv2.imread("/home/lustbeast/office/Tamannaah_at_an_event_in_Cochin,_July_2018.jpg")
mtcnn = MTCNN(image_size=256,device="cuda:0")
boxes,prob = mtcnn.detect(img)
img = extract_face(img,box=bo)

IndexError: index 2 is out of bounds for axis 0 with size 1

In [188]:
out = model(img)

In [189]:
torch.sigmoid(out[0])

tensor([[0.2742]], grad_fn=<SigmoidBackward>)

In [191]:
out[1]

tensor([[0.7519]], grad_fn=<SigmoidBackward>)